In [ ]:
#Word cloud generation derived from notebook of Omdena ACET Challenge(https://omdena.com/blog/infrastructural-needs/)
%%capture
import re
import pandas as pd

In [ ]:
'''Another issue to consider when working with social media data is how to deal with emojis, 
mentions (@MisterX), hashtags (#Africa), email addresses, and URLs. 
The solution depends on your specific task. 
In our project, we used Python re module to remove emojis, email addresses, URLs, and hash symbol 
so that for instance, “#Africa” becomes “Africa”.'''

In [ ]:
emoji_pattern = re.compile("["
   u"\U0001F600-\U0001F64F" # emoticons
   u"\U0001F300-\U0001F5FF" # symbols & pictographs
   u"\U0001F680-\U0001F6FF" # transport & map symbols
   u"\U0001F1E0-\U0001F1FF" # flags (iOS)
   u"\U00002702-\U000027B0"
   u"\U000024C2-\U0001F251"
   u"\U00002500-\U00002BEF" # chinese char
   u"\U0001f921-\U0001f937"
   u"\U00010000-\U0010ffff"
   u"\u2640-\u2642"
   u"\u2600-\u2B55"
   u"\u200d"
   u"\u23cf"
   u"\u23e9"
   u"\u231a"
   u"\ufe0f" # dingbats
   u"\u3030"
   "]+", flags=re.UNICODE)
email_pattern = re.compile("\S+@\S+\.\S{2,3}")
link_pattern = re.compile("https?\S+")

mentions_pattern = re.compile("@[a-zA-Z0-9_.]+?(?![a-zA-Z0-9_.])")
#mentions_pattern = re.compile("(?<![^\s])(([@]{1}|[#]{1})[A-Za-z0-9]+)(?![^\s])")

#numbers_pattern = re.compile("gsub('[[:digit:]]+', '', x)")
numbers_pattern = re.compile('[0-9]+')
#re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df= pd.read_csv('/content/twint_scraped_2022_May_Jul.csv')
df.drop(df.columns.difference(['id','created_at', 'username', 'tweet', 'hashtags']), 1, inplace=True)
#df.to_csv('twint_scraped_2021_5cols.csv')

In [ ]:
def clean_data(tweet):
    try:
        tweet_rep = emoji_pattern.sub(r'', tweet)
        tweet_rep = email_pattern.sub(r'', tweet_rep)
        tweet_rep = link_pattern.sub(r'', tweet_rep)

        tweet_rep = mentions_pattern.sub(r'', tweet_rep)

        tweet_rep = numbers_pattern.sub(r'', tweet_rep)

        tweet_rep = tweet_rep.replace("’", "‘")
        tweet_rep = tweet_rep.replace("&amp;", "&")
        tweet_rep = tweet_rep.replace("#", '')
        tweet_rep = tweet_rep.strip()
        return tweet_rep
    except:
        return ""

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
clean_tweet_list=[]
for tweet in df.tweet:
    #print(i)
    clean_tweet=clean_data(tweet)
    clean_tweet_list.append(clean_tweet)
print(clean_tweet_list)

In [ ]:
print(len(clean_tweet_list))

In [ ]:
clean_tweet_list_low = [el.lower() for el in clean_tweet_list]
print(clean_tweet_list_low)

In [ ]:
df2 = pd.read_csv('/content/stopwords-it_final.txt', delimiter = "\t", names = ['colA'])

In [ ]:
stopwords_iso = set(df2['colA'])

In [ ]:
print(len(list(stopwords_iso)))
print(sorted(list(stopwords_iso)))

In [ ]:
import spacy

In [ ]:
!python -m spacy download it_core_news_sm
#!python -m spacy download it_core_news_lg

In [ ]:
# Initialize spacy 'it_core_news_sm' model, keeping only tagger component (for efficiency)
#!python -m spacy download it_core_news_sm
nlp = spacy.load('it_core_news_sm', disable=['parser', 'ner'])
#nlp = spacy.load('it_core_news_sm')
#nlp = spacy.load('it_core_news_lg')

In [ ]:
import nltk
nltk.download('stopwords')
it_stop_nltk = nltk.corpus.stopwords.words('italian')

In [ ]:
print(type(it_stop_nltk))
print(len(list(it_stop_nltk)))
print(sorted(list(it_stop_nltk)))

it_stop_nltk = set(it_stop_nltk)

In [ ]:
from spacy.lang.it.stop_words import STOP_WORDS as it_stop
print(type(it_stop))
print(len(list(it_stop)))  # #=624
print(sorted(list(it_stop)))

it_stop |= it_stop_nltk

print(len(list(it_stop)))
print(sorted(list(it_stop)))

In [ ]:
it_stop |= stopwords_iso

print(len(list(it_stop)))
print(sorted(list(it_stop)))

In [ ]:
it_stop |= {"covid19","covid", "coronavirus", "covid-19", "long", "longcovid", "virus", "vaccino", "vaccini", "vaccinati","vaccinazione", "vaccinato", "italia", "sarscov2", "sars", "cov","ah","eh","oh","uh","ehi"}

print(len(list(it_stop)))
print(sorted(list(it_stop)))

In [ ]:
def tweetBatch_to_words(sentences):
    for sentence in sentences:
      doc = nlp(sentence)
      for token in doc:
#        yield token.text, token.is_stop, token.is_punct
        yield token.text, token.text in it_stop, token.is_punct

In [ ]:
import math

n = math.ceil(len(clean_tweet_list_low)/4000)
print(n)

In [ ]:
#tokens = list(tweetBatch_to_words(clean_tweet_list))  #ERROR IOPub data rate exceeded.

#due to the above error, split of clean_tweet_list to yield tokens
tokens = []

if n > 1:
  for i in range(n-1):
    tokens.append(list(tweetBatch_to_words(clean_tweet_list_low[i*4000:(i+1)*4000])))

  tokens.append(list(tweetBatch_to_words(clean_tweet_list_low[(n-1)*4000:])))
else:
  tokens.append(list(tweetBatch_to_words(clean_tweet_list_low)))


In [ ]:
cleaned = []
for i in range(n):
  cleaned += [el[0] for el in tokens[i] if not el[1] and not el[2] and not el[0].isspace()]

In [ ]:
print(len(cleaned))
print(cleaned)

In [ ]:
# join in unique text required for wordcloud
text = ' '.join([str(el) for el in cleaned])
print(text)

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt
% matplotlib inline

#wordcloud = WordCloud().generate(text)

wordcloud = WordCloud(
    background_color = 'white', 
    width = 1600, height = 800,
    max_font_size = 100,
    min_font_size = 10, 
    random_state = 0,
    stopwords = it_stop,
    scale = 1,
    max_words = 500,
    collocations = True   #Whether to include collocations (bigrams) of two words. 
).generate(text)

plt.figure(figsize=[14,18])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
#plt.title('top 500 unigrams-bigrams frequencies from January to July 2022 LC tweets')

In [ ]:
wordcloud.to_file('twint_scraped_2022_May_Jul.png')

In [ ]:
wordcloud.words_

In [ ]:
import json
with open('tokens_freq_2022_May_Jul.json', 'w') as fp:
    json.dump(wordcloud.words_, fp)

In [ ]:
print(len(wordcloud.words_))
wordcloud.words_['miocarditi']

<b>Wordcloud from tf-idf frequencesies.</b>

In [ ]:
print(len(list(it_stop)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(stop_words=list(it_stop), ngram_range=(1,2)) # instantiate an object from the TfidfVectorizer class (constructer function)
vec.fit(clean_tweet_list_low) # fit the object to our data

df_tfidf= pd.DataFrame(vec.transform(clean_tweet_list_low).toarray(), columns=sorted(vec.vocabulary_.keys()))

In [ ]:
#df_tfidf.tail()

In [ ]:
#df_tfidf.T.sum(axis=1)

In [ ]:
wordcloud_tfidf = WordCloud(
    background_color = 'white', 
    width = 1600, height = 800,
    max_font_size = 100,
    min_font_size = 10, 
    random_state = 0,
    stopwords = it_stop,
    scale = 1,
    max_words = 500,
    collocations = True   #Whether to include collocations (bigrams) of two words. 
    ).generate_from_frequencies(df_tfidf.T.sum(axis=1))

plt.figure(figsize=[14,18])
plt.imshow(wordcloud_tfidf, interpolation='bilinear')
plt.axis("off")

In [ ]:
wordcloud_tfidf.to_file('twint_scraped_2022_May_Jul_tfidf.png')

In [ ]:
wordcloud_tfidf.words_

In [ ]:
import json
with open('tokens_freq_2022_May_Jul_tfidf.json', 'w') as fp:
    json.dump(wordcloud_tfidf.words_, fp)

In [ ]:
wordcloud_tfidf.words_['miocarditi']

<b>Extract keywords with RAKE method</b>

In [ ]:
!pip install rake-nltk

In [ ]:
from rake_nltk import Rake
import string
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

#r = Rake(stopwords=it_stop, language='italian',max_length=2, include_repeated_phrases=False,punctuations=string.punctuation)
#r = Rake(stopwords=it_stop, language='italian',max_length=2,punctuations=string.punctuation, sentence_tokenizer=tknzr)
r = Rake(stopwords=it_stop, language='italian',max_length=2,punctuations=string.punctuation)


# Extraction given the text.
#r.extract_keywords_from_text(text)

# Extraction given the list of strings where each string is a sentence.
r.extract_keywords_from_sentences(clean_tweet_list_low)

# To get keyword phrases ranked highest to lowest.
#r.get_ranked_phrases()

# To get keyword phrases ranked highest to lowest with scores.
results = r.get_ranked_phrases_with_scores()


In [ ]:
print(len(results))
print(results)

In [ ]:
results_fin = [(ele1, ele2) for (ele1, ele2) in results if ele1 >= 2.0]
print(len(results_fin))
print(results_fin)

In [ ]:
di = {}
for (score, keyword) in results_fin:
  di[keyword] = score
di

In [ ]:
import json
with open('tokens_freq_2022_May_Jul_RAKE.json', 'w') as fp:
    json.dump(di, fp)

In [ ]:
key_check = 'nebbia cerebrale'
if(key_check in di.keys()):
  print(di[key_check])